In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt


Task: Build a Transfer Learning model to detect face masks on humans.

Task A:
•Load the Image Training and Test Datasets from the train and test folders respectively. The 
size of each image is 128 x 128 x 3

In [ ]:
train_file_path = '/home/cullen-fedora/Documents/With_mask_1'

In [ ]:
test_file_path = '/home/cullen-fedora/Documents/without_mask_1'

•Load training dataset using Keras ImageDataGenerator with validation_split=0.2  

•Load test dataset using Keras ImageDataGenerator 
Build a Transfer Learning network using Keras with the following layers:

In [ ]:
# Define paths to your dataset
train_directory = train_file_path
test_directory = test_file_path

# Image dimensions
image_size = (128, 128)  # Size of each image
batch_size = 32           # Adjust batch size as needed

# Set up ImageDataGenerator for training data with augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,       # Normalize pixel values to [0, 1]
    validation_split=0.2     # Specify validation split
)

# Load training data
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=image_size,   # Resize images
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class; 'binary' for binary classification
    subset='training',         # Set as training data
    seed=42                   # Ensure reproducibility
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=image_size,   # Resize images
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-class
    subset='validation',       # Set as validation data
    seed=42                   # Ensure reproducibility
)

# Set up ImageDataGenerator for test data (no validation split needed)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Only rescale for test data

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=image_size,    # Resize images
    batch_size=batch_size,
    class_mode='categorical',   # Multi-class expected
    shuffle=False               # Don't shuffle test data
)

•Load EfficientNetB0 as first layers using Keras API

•GLobalAveragePooling2D layer

•Dropout (0.2)

•Dense layer with 3 neurons and activation SoftMax


In [ ]:
# Load EfficientNetB0 without top layers (include_top=False)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Build the model using the Sequential API
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Global average pooling to flatten the output
    Dropout(0.2),              # Dropout for regularization
    Dense(1, activation='softmax')  # Output layer for 3 classes (with_mask, without_mask, and possibly a third category)
])

Compile the model with Adam optimizer, categorical_crossentropy loss and with metrics 
accuracy

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

•Train the model for 25 epochs with callbacks Reduce Learning Rate on Plateau and early stopping while monitoring validation loss

In [ ]:
# Callbacks: Reduce Learning Rate on Plateau and Early Stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=25,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[reduce_lr, early_stopping]
    )

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'\nTest accuracy: {test_accuracy:.2f}')

•Plot training and validation accuracy and loss against epochs


In [ ]:
# Plotting the training and validation accuracy and loss
def plot_training_history(history):
    # Training and validation accuracy
    plt.figure(figsize=(12, 4))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Call the plotting function
plot_training_history(history)

Task B
•Load the Image Training and Test Datasets from the train and test folder respectively. 
The size of each image is 128 x 128 x 3

•Load training dataset using Keras ImageDataGenerator with validation_split=0.2  

•Load test dataset using Keras ImageDataGenerator 
Build a Transfer Learning network using Keras with the following layers:

•Load ResNet50 as first layers using Keras API

•GLobalAveragePooling2D layer

•Dropout(0.5)

•Dense layer with 3 neurons and activation SoftMax


•Compile the model with Adam optimizer,categorical_crossentropy loss and with metrics 
accuracy.

•Train the model for 25 epochs with callbacks Reduce Learning Rate on Plateau and early stopping while monitoring validation loss

•Plot training and validation accuracy and loss against epochs

In [ ]:
# Load datasets
train_datagen = ImageDataGenerator(validation_split=0.2, rescale=1.0/255.0)
train_generator = train_datagen.flow_from_directory(
    'train',                # Directory with training data
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'train',                # Same directory as training data
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    'test',                 # Directory with testing data
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# Build the model
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=25,
    callbacks=[reduce_lr, early_stopping]
)

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

Task C
•Compare EfficientNetB0 and ResNet50 model performance and find the best model

•Using the best model, predict the test dataset and plot 10 images from the test set along with its True Label and Predicted Label

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load test dataset
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    'test',                 # Directory with testing data
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False          # Important to retain the order of the images for predictions
)

# Evaluate EfficientNetB0 model
efficientnetb0_eval = model.evaluate(test_generator)
print("EfficientNetB0 Performance:")
print(f"Loss: {efficientnetb0_eval[0]}; Accuracy: {efficientnetb0_eval[1]}")

# Assume `model_resnet50` is your ResNet50 model built previously
resnet50_eval = model_resnet50.evaluate(test_generator)
print("ResNet50 Performance:")
print(f"Loss: {resnet50_eval[0]}; Accuracy: {resnet50_eval[1]}")

# Determine the better model based on accuracy
best_model = model if efficientnetb0_eval[1] > resnet50_eval[1] else model_resnet50
best_model_name = "EfficientNetB0" if efficientnetb0_eval[1] > resnet50_eval[1] else "ResNet50"
print(f"The best model is: {best_model_name}")

# Predict on the test dataset using the best model
test_generator.reset()  # Reset the generator for prediction
predictions = best_model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Plot 10 images with true and predicted labels
plt.figure(figsize=(15, 10))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(test_generator[i][0][0])
    plt.axis('off')
    plt.title(f'True: {class_labels[true_classes[i]]}\nPred: {class_labels[predicted_classes[i]]}')

plt.tight_layout()
plt.show()